<a href="https://colab.research.google.com/github/Ankitarora2/TechnoCulture/blob/main/AMIESignatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serve the model using vLLM

In [2]:
!pip install dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [3]:
# Run server in foreground
# !python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq

# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &
# stdout is redirected to a file `server.log` using `> server.log`.
# We use a quantized model prepared using AWQ quantization

In [11]:
# Run this cell again and again to monitor the status of the server.
# The server can take a few mintues to start.
# Once the server has started, you will see logs such as this:
# INFO 02-10 07:16:43 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
!tail server.log

INFO 02-26 12:17:29 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:17:39 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:17:49 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:17:59 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
INFO 02-26 12:18:09 metrics.py:161] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapp

In [12]:
# Once the server is up and running, this should work
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","object":"model","created":1708949943,"owned_by":"vllm","root":"TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ","parent":null,"permission":[{"id":"modelperm-da2e39ff6119457a829dd7b8a1c141d6","object":"model_permission","created":1708949943,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

# DSPy: 𝗗eclarative 𝗦elf-improving Language 𝗣rograms

In [13]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [14]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

In [15]:
predict = dspy.Predict('question -> answer')

predict(question="What is the capital of Germany?")

Prediction(
    answer='Berlin'
)

# **DSPY.SIGNATURES**

In [53]:
class GMCPQ(dspy.Signature):
  """ How would you rate the doctor at the following criteria, on a 5 point scale. Poor Less than Satisfactory Satisfactory Good Very Good Cannot rate / Does not apply"""

conversation= dspy.InputField(desc="conversation between doctor and patient")
#context= dspy.InputField(desc="Criteria: Being polite")
ratings= dspy.OutputField(desc="""ratings of doctor on criteria: being polite""")


In [ ]:
Being polite;
Making you feel at ease;
 Listening to you;
Assessing your medical condition;
Explaining your condition and treatment;
Involving you in decisions about your treatment;
Providing or arranging treatment for you

In [54]:
convo="""Patient – Good afternoon, Doctor.

Doctor – Good afternoon, Mr. Bose. How are you?

Patient – I’m doing good, doctor, but my daughter isn’t doing well. Everywhere, people are getting affected with COVID and I am really worried about her.

Doctor – Please have a seat and tell me what happened.

Patient – Last week, my daughter came back from Pune as her college was closed on account of COVID. From the second day, she has had high fever and has been coughing badly. I think that she has contracted the virus on her way home.

Doctor – Okay, I understand your concern. Having a fever and cough doesn’t necessarily mean that someone has contracted the virus. These are symptoms of common cold too. The change in the temperature of the atmosphere could have triggered these symptoms. Still, to put your worries to rest, I am prescribing some medicines and an RT PCR test. Do the test by tomorrow, and if the test results are positive, make sure she is isolated. On the other hand, if the result is negative, just give her the medicine and ask her to drink a lot of water. Also, bring her in so I could examine her.

Patient – Okay, doctor. I will bring her in the evening. Thank you.

Doctor – You are welcome."""

In [56]:
judge = dspy.ChainOfThought(signature=GMCPQ, )
judge(conversation=convo)

ValueError: not enough values to unpack (expected at least 1, got 0)

In [ ]:
class search_retrieval(dspy.Signature):
  """What are the specific patient demographics/symptoms/management plan for the condition"""

condition = dspy.InputField(desc=“medical condition”)
passage=dspy.OutputField(desc=”a passage including specific patient demographics/symptoms/management plan”)


#Search Retrieval

`What are the specific patient demographics/symptoms/management plan for the condition [Condition]?`

In [ ]:
"""What are the specific patient demographics/symptoms/management plan for the condition?"""
#INPUT
condition = dspy.InputField(desc="medical condition")

#OUTPUT
passage=dspy.OutputField(desc="a passage including specific patient demographics/symptoms/management plan")

#Passage Filtering Template



```
For the clinical condition, [Condition], is the following a good description of common demographics/symptoms/management plans (Yes/No)?
Description: [Retrieved Passage]
Answer (Yes/No):
```



In [ ]:
"""For the clinical condition, is the following a good description of common demographics/symptoms/management plans (Yes/No)?"""
#INPUT
condition = dspy.InputField(desc="medical condition")
passage=dspy.InputField(desc="a passage including specific patient demographics/symptoms/management plan")

#OUTPUT
is_good = dspy.OutputField(desc="Yes/No")

#Vignette Generation Template




```
The following are several passages about the demographics, symptoms, and management plan for a given condition. Generate 2 different patient vignettes consistent with these passages. Follow the format of the given example (just list N/A if a particular field is unavailable).
Condition: [Condition]
Demographic Passages: [Retrieved Demographic Passages]
Symptoms Passages: [Retrieved Symptom Passages]
Management Plan Passages: [Retrieved Management Plan Passages]
Example Format: [Oneshot example]
Patient Vignettes for [Condition]:
```



In [ ]:
"""The following are several passages about the demographics, symptoms, and management plan for a given condition. Generate 2 different patient vignettes consistent with these passages. Follow the format of the given example (just list N/A if a particular field is unavailable)"""
#INPUT
condition = dspy.InputField(desc="medical condition")

demographic_passages= dspy.InputField(desc="[Retrieved Demographic Passages]")

symptom_passages= dspy.InputField(desc="[Retrieved Symptom Passages]")

management_passages= dspy.InputField(desc="[Retrieved Management Passages]")

example = dspy.InputField(desc="[Oneshot example]")

#OUTPUT
vignette=dspy.OutputField(desc="2 different patient Vignettes for [MedicalCondition]:)")


# Patient Agent



```
You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. <patient vignette> Respond to the doctor's questions honestly as they interview you, asking any questions that may come up.
```



In [ ]:
"""You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. <patient vignette> Respond to the doctor's questions honestly as they interview you, asking any questions that may come up."""

#INPUT
vignette=dspy.InputField(desc="patient vignette")

doctors_question=dspy.InputField(desc="doctor's question")

#OUTPUT
patients_response=dspy.OutputField(desc="Response to the doctor's questions, along with any questions that may come up.")


#ALTERNATIVE OUTPUT
patients_response=dspy.OutputField(desc="Response to the doctor's questions.")
patients_question=dspy.OutputField(desc="Any questions that may come up.")

# Doctor Agent

`You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately.`

In [ ]:
"""You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."""

#INPUT
patients_response=dspy.InputField(desc="patient's response")

#OUTPUT
doctors_response=dspy.OutputField(desc="Response to the patient's questions, along with any questions that may come up.")


#ALTERNATIVE OUTPUT
response=dspy.OutputField(desc="Response to the patient's questions.")
doctors_question=dspy.OutputField(desc="Any questions that may come up. Do not ask more than 2 questions.")


#ASSERT SUGGEST
"""Respond to the patient with a single-turn response to better understand their history and symptoms.
Do not ask more than two questions."""

Self Play Critic

In [ ]:
""" Act as a critic aware of the ground truth diagnosis, to provide in-context feedback to the doctor agent for selfimprovement which can enhance its performance"""
#INPUT

ground_truth=dspy.InputField(desc="ground truth")
preeceding_conversation=dspy.InputField(desc="preeceding conversation")
doctors_response=dspy.InputField(desc="doctor's response")

#OUTPUT
empathy = dspy.OutputField(desc="""Rating(0 to 5) of the doctor's response based on if the doctor agent exhibits empathy and professionalism while addressing the patient’s latest questions or comments in a concise manner.
0: Complete lack of empathy and professionalism, responses are dismissive or unhelpful.
1: Minimal empathy and professionalism, responses are brief and lack warmth or understanding.
2: Some empathy and professionalism, responses are moderately concise but lack depth or personalization.
3: Good empathy and professionalism, responses are sufficiently concise and demonstrate understanding.
4: High level of empathy and professionalism, responses are very concise yet convey genuine concern and understanding.
5: Excellent empathy and professionalism, responses are exceptionally concise, demonstrating deep empathy and professionalism.""")

repititve_questions=dspy.OutputField(desc="""The doctor avoids asking too many or repetitive questions (about information already acquired), focusing on a maximum of one or two questions per response.(Yes/No)""")

flow_naturally=dspy.OutputField(desc="""Rating(0 to 5) of the doctor's response on the follwing criteria: Flows naturally and consistently, with or without interruptions.
0: Responses consistently feel disjointed and unnatural, making conversation difficult.
1: Responses often feel forced or robotic, disrupting the natural flow of conversation.
2: Responses sometimes flow naturally, but noticeable awkwardness or stiffness hampers the interaction.
3: Responses generally flow naturally, allowing for smooth conversation with occasional minor disruptions.
4: Responses mostly flow naturally, resembling human conversation with minimal interruptions or awkwardness.
5: Responses seamlessly flow naturally, mirroring human conversation perfectly, enhancing engagement.""")

factual_accuracy=dspy.OutputField(desc="""Rating(0 to 5) of the doctor's response on the follwing criteria: Maintain Factual Accuracy.
0: Responses frequently contain significant inaccuracies or misinformation.
1: Responses often provide inaccurate or misleading information.
2: Responses occasionally contain factual errors, undermining trust and reliability.
3: Responses generally maintain factual accuracy, with occasional minor inaccuracies.
4: Responses mostly provide accurate information, enhancing trust and reliability.
5: Responses consistently maintain impeccable factual accuracy, instilling confidence in the information provided.""")

further_engagement=dspy.OutputField(desc="""Rating(0 to 5) of the doctor's response on the follwing criteria: Facilitate Further Engagement from the Patient.
0: Responses consistently fail to encourage further interaction or engagement.
1: Responses often do not prompt further engagement or participation from the patient.
2: Responses occasionally prompt further interaction but lack effectiveness in engaging the patient.
3: Responses generally encourage further engagement, fostering continued interaction from the patient.
4: Responses effectively facilitate further engagement, prompting meaningful interaction from the patient.
5: Responses consistently and skillfully encourage ongoing engagement, maximizing patient participation and interaction..""")




# Moderator Agent



```
The following is a conversation between a doctor and a patient: <dialog> The conversation should only come to an end if the doctor has finished giving the patient a diagnosis and treatment plan and the patient has no questions left. A conversation also comes to an end if the doctor or patient says goodbye.
Question: has the conversation come to an end? Yes or No.
```







In [ ]:
"""You're acting as a moderator for a conversation between a doctor and a patient. The conversation should only come to an end if the doctor has finished giving the patient a diagnosis and treatment plan and the patient has no questions left. A conversation also comes to an end if the doctor or patient says goodbye."""
#INPUT
conversation=dspy.InputField(desc="conversation between a doctor and a patient")

#OUTPUT
end_of_conversation=dspy.OutputField(desc="Yes or No")

# Chain of Reasoning For Inference



```
1. Analyzing patient information: Given the current conversation history, AMIE was instructed to
1)
summarize the positive and negative symptoms of the patient as well as any relevant medical/family/social
history and demographic information,
2) produce a current differential diagnosis,
3) note missing
information needed for a more accurate diagnosis and
4) assess confidence in the current differential and
highlight its urgency.
2. Formulating response and action: Building upon the conversation history and the output of step 1, AMIE performed the following:
1) Generate a response to the patient’s last message and formulate
further questions to acquire missing information and refine the differential diagnosis.
2) If necessary,
recommend immediate action, such as an emergency room visit. If confident in the diagnosis based on
available information, present the differential.
3. Refining the response: AMIE revises its previous output to meet specific criteria based on the
conversation history and outputs from earlier steps. The criteria are primarily related to factuality and
formatting of the response (e.g., avoid factual inaccuracies on patient facts and unnecessary repetition,
show empathy, and display in a clear format).

```



Patient Vignette through conversation.

In [ ]:
"""Summarize the positive and negative symptoms of the patient as well as any relevant medical/family/socialhistory and demographic information"""
#INPUT
conversation=dspy.InputField(desc="conversation between a doctor and a patient")

#OUTPUT
summarized_symptoms=dspy.OutputField(desc="patient vignette")

DDx

In [ ]:
current_ddx=dspy.OutputField(desc="differntial diagnosis based upon the given patient vignette")

Missing Information

In [ ]:
missing_information=dspy.OutputField(desc="missing information needed for a more accurate diagnosis")

Confidence in Diagnosis and Urgency

In [ ]:
confidence=dspy.OutputField(desc="""Assess confidence in the given diagnosis. Based upon this scale, provide a rating from 0 to 5.

0: No confidence - There is no confidence in any of the provided potential diagnoses.
1: Very low confidence - There is minimal confidence in the provided potential diagnoses, with significant uncertainty.
2: Low confidence - There is some confidence in the provided potential diagnoses, but with notable reservations or uncertainty.
3: Moderate confidence - There is moderate confidence in the provided potential diagnoses, with a reasonable level of certainty.
4: High confidence - There is high confidence in the provided potential diagnoses, with strong conviction and minimal doubt.
5: Very high confidence - There is very high confidence in the provided potential diagnoses, with unwavering certainty and strong supporting evidence.""")

urgency=dspy.OutputField(desc="""Assess urgency of the given diagnosis. Rate as Yes if the patient is seeking urgent care. Yes/No""")

Formulating response and action

In [ ]:
"""Generate a response to the patient’s last message and formulate
further questions to acquire missing information and refine the differential diagnosis."""

#INPUT
conversation=dspy.InputField(desc="conversation between a doctor and a patient")
ddx=dspy.InputField(desc="differential diagnosis based upon the given patient vignette")
missing_information=dspy.InputField(desc="missing information needed for a more accurate diagnosis")
urgency=dspy.InputField(desc="""Assess urgency of the given diagnosis. Rate as Yes if the patient is seeking urgent care. Yes/No""")

#OUTPUT
response=dspy.OutputField(desc="""Response to the patient's last message along with further questions,
If necessary, recommend immediate action, such as an emergency room visit.
If confident in the diagnosis based on available information, present the differential.""")

Refining the response(RECHECK)

In [ ]:
"""Refine the response conditioned on the current conversation to arrive at an informed and grounded reply. Avoid factual inaccuracies on patient facts and unnecessary repetition, while showing empathy, and display in a clear format"""
#INPUT
conversation=dspy.InputField(desc="conversation between a doctor and a patient")
response=dspy.InputField(desc="response to the patient's last message")

#OUTPUT
refined_response=dspy.OutputField(desc="""Refined response to the patient's last message along with further questions if required.""")

# GMCPQ

Assessed by Patient Actor

In [ ]:
"""You're supposed to answer as the patient in this conversation. How would you rate the doctor at the given criteria:"""

#INPUT
conversation=dspy.InputField(desc="conversation between a doctor and a patient")


#Being Polite
dspy.InputField(desc="""Criteria: Being Polite.""")

#Making you feel at ease
dspy.InputField(desc="""Criteria:Making you feel at ease.""")

#Listening to you
dspy.InputField(desc="""Criteria: Listening to you.""")

#Assessing your medical condition
dspy.InputField(desc="""Criteria: Assessing your medical condition.""")

#Explaining your condition and treatment
dspy.InputField(desc=""" Criteria: Explaining your condition and treatment.""")

#Involving you in decisions about your treatment
dspy.InputField(desc="""Criteria: Involving you in decisions about your treatment.""")

#Reassuring you of your medical condition
dspy.InputField(desc="""Criteria: Reassuring you of your medical condition.""")

#Providing or arranging treatment for you
dspy.InputField(desc="""Criteria: Providing or arranging treatment for you.""")


#OUTPUT RATING
dspy.OutputField(desc=""" Rating based on a 5 point scale for the given criteria:
0:Cannot rate / Does not apply
1:Poor,
2:Less than Satisfactory,
3:Satisfactory,
4:Good,
5:Very Good
""")

In [ ]:
#This doctor will keep information about me confidential
dspy.InputField(desc="How much do you agree with the following statement? This doctor will keep information about me confidential.")
dspy.OutputField(desc="""Rating based on a 5 point scale:
0:Cannot rate / Does not apply
1:Strongly disagree
2:Disagree
3:Neutral
4:Agree
5:Strongly agree""")

#This doctor is honest and trustworthy.
dspy.InputField(desc="How much do you agree with the following statement? This doctor is honest and trustworthy.")
dspy.OutputField(desc="""Rating based on a 5 point scale:
0:Cannot rate / Does not apply
1:Strongly disagree
2:Disagree
3:Neutral
4:Agree
5:Strongly agree""")

#I am confident about this doctor's ability to provide care.
dspy.InputField(desc="How much do you agree with the following statement? I am confident about this doctor's ability to provide care.")
dspy.OutputField(desc="""Rating based on a binary scale:
Cannot rate / Does not apply
0:No
1:Yes""")

#I would be completely happy to see this doctor again.
dspy.InputField(desc="How much do you agree with the following statement? I would be completely happy to see this doctor again.")
dspy.OutputField(desc="""Rating based on a binary scale:
Cannot rate / Does not apply
0:No
1:Yes""")

# PACES

In [ ]:
"""You're supposed to answer as a specialist physician who is tasked to assess the doctor in this conversation. How would you rate the doctor at the given criteria on a scale of 0 to 5:"""

#INPUT
conversation=dspy.InputField(desc="conversation between a doctor and a patient")

#CRITERIA
dspy.InputField(desc="Criteria:To what extent did the doctor elicit the PRESENTING COMPLAINT?")

dspy.InputField(desc="Criteria:To what extent did the doctor elicit the SYSTEMS REVIEW? ")

dspy.InputField(desc="Criteria:To what extent did the doctor elicit the FAMILY HISTORY?")

dspy.InputField(desc="Criteria:To what extent did the doctor elicit the MEDICATION HISTORY?")

#OUTPUT

dspy.OutputField(desc=""" Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Appears unsystematic, unpracticed, and unprofessional
2: Shows some effort but lacks thoroughness and fluency; displays occasional professionalism.
3: Demonstrates moderate systematic approach and professionalism, though with some room for improvement.
4: Exhibits a mostly systematic, fluent, and professional approach, with occasional minor lapses.
5: Elicits the criteria in a thorough, systematic, fluent, and professional manner

""")

In [ ]:
#To what extent did the doctor explain relevant clinical information ACCURATELY?
dspy.InputField(desc="Criteria:To what extent did the doctor explain relevant clinical information ACCURATELY?")



#OUTPUT

dspy.OutputField(desc=""" Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Gives inaccurate information
2: Provides information with some inaccuracies or inconsistencies, requiring clarification.
3: Offers generally accurate information but may have occasional errors or omissions.
4: Presents mostly accurate information with few errors, demonstrating overall reliability.
5: Explains relevant clinical information accurately and comprehensively.
""")

In [ ]:
#To what extent did the doctor explain relevant clinical information CLEARLY?

#INPUT
dspy.InputField(desc="Criteria:To what extent did the doctor explain relevant clinical information CLEARLY?")



#OUTPUT

dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Uses excessive jargon, impeding comprehension.
2: Occasionally relies on jargon, causing some confusion but generally understandable.
3: Balances jargon with clear explanations, promoting reasonable understanding.
4: Rarely employs jargon, preferring clear language for effective communication.
5: Provides clear and easily understandable explanations of relevant clinical information.
""")

In [ ]:
#To what extent did the doctor explain relevant clinical information WITH STRUCTURE?

#INPUT
dspy.InputField(desc="Criteria:To what extent did the doctor explain relevant clinical information WITH STRUCTURE?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0:Cannot rate / Does not apply / Doctor did not perform this
1:Explains relevant clinical information in a poorly structured manner, lacking coherence or organization.
2:Provides some structure to the explanation but may be disorganized or lacking in clarity.
3:Offers moderate structure to the explanation, with some organization but room for improvement.
4:Presents clinical information in a structured manner, with clear organization facilitating understanding.
5:Explains relevant clinical information in a highly structured manner, making it easy to follow and comprehend.
""")

In [ ]:
#To what extent did the doctor explain relevant clinical information COMPREHENSIVELY?

#INPUT
dspy.InputField(desc="Criteria:To what extent did the doctor explain relevant clinical information WITH STRUCTURE?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Omits important information, leaving significant gaps in understanding.
2: Provides some relevant information but overlooks key details, reducing overall comprehensiveness.
3: Offers moderate coverage of relevant clinical information, but may lack depth or thoroughness.
4: Presents a comprehensive overview of relevant clinical information, addressing most important aspects.
5: Explains relevant clinical information in a highly comprehensive manner, covering all pertinent details thoroughly.
Cannot rate / Does not apply / Doctor did not perform this
""")

In [ ]:
#To what extent did the doctor explain relevant clinical information PROFESSIONALLY?

#INPUT
dspy.InputField(desc="Criteria:To what extent did the doctor explain relevant clinical information PROFESSIONALLY?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0:Cannot rate / Does not apply / Doctor did not perform this
1: Explains relevant clinical information in an unprofessional manner
2: Provides relevant clinical information with occasional unprofessionalism or lack of polish.
3: Conveys relevant clinical information in a generally professional manner, with occasional minor lapses.
4: Explains relevant clinical information in a consistently professional manner, demonstrating professionalism throughout.
5: Consistently and adeptly explains relevant clinical information in a highly professional manner.
""")

Differential Diagnosis

In [ ]:
#To what extent did the doctor select a comprehensive, sensible and appropriate MANAGEMENT PLAN?


#INPUT
dspy.InputField(desc="Criteria:To what extent did the doctor select a comprehensive, sensible and appropriate MANAGEMENT PLAN?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Poor differential diagnosis AND fails to consider the correct diagnosis
2: Constructs a differential diagnosis, but with notable gaps or inaccuracies, possibly missing the correct diagnosis.
3: Develops a reasonably sensible differential diagnosis, though with some omissions or minor inaccuracies.
4: Constructs a coherent and logical differential diagnosis, addressing most relevant possibilities, with minor inaccuracies.
5: Constructs a sensible differential diagnosis, including the correct diagnosis among other relevant possibilities.
""")


Clinical Judgement


In [ ]:
#To what extent did the doctor construct a sensible DIFFERENTIAL DIAGNOSIS?

#INPUT
dspy.InputField(desc="Criteria:To what extent did the doctor construct a sensible DIFFERENTIAL DIAGNOSIS?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Unfamiliar with correct management plan AND selects inappropriate management
2: Selects a management plan with significant gaps or inaccuracies, possibly inappropriate.
3: Develops a management plan that is reasonably comprehensive and sensible, with some room for improvement.
4: Constructs a coherent and logical management plan, addressing most relevant aspects, with minor inaccuracies.
5: Selects a comprehensive, sensible, and appropriate management plan, demonstrating thorough understanding and application of best practices.
""")


Managing Patient Concerns

Assessed By Specialist &
Patient Actor

In [ ]:
#To what extent did the doctor seek, detect, acknowledge, and attempt to address the patient's concerns?

#INPUT
dspy.InputField(desc="Criteria: To what extent did the doctor seek, detect, acknowledge, and attempt to address the patient's concerns?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Overlooks patient's concerns
2: Shows minimal effort to address patient's concerns, with little acknowledgment or attempt at resolution.
3: Acknowledges some patient concerns and makes an attempt to address them, but with room for improvement.
4: Demonstrates a proactive approach in seeking, detecting, acknowledging, and attempting to address patient's concerns, with occasional minor oversights.
5: Seeks, detects, acknowledges, and attempts to address patient's concerns comprehensively and effectively, fostering patient satisfaction and trust.
""")


In [ ]:
#To what extent did the doctor confirm the patient's knowledge and understanding?

#INPUT
dspy.InputField(desc="Criteria: To what extent did the doctor confirm the patient's knowledge and understanding?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Does not check knowledge and understanding
2: Makes minimal effort to confirm patient's knowledge and understanding, with limited effectiveness.
3: Makes some attempt to confirm patient's knowledge and understanding, but with room for improvement.
4: Demonstrates a satisfactory effort in confirming patient's knowledge and understanding, with occasional oversights.
5: Confirms patient's knowledge and understanding effectively and comprehensively, ensuring clarity and comprehension.
""")


In [ ]:
#How empathic was the doctor?

#INPUT
dspy.InputField(desc="Criteria: How empathic was the doctor?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Not at all empathic
2: Shows minimal empathy, lacking in understanding or concern for patient's emotions.
3: Demonstrates moderate empathy, displaying some understanding and acknowledgment of patient's emotions.
4: Exhibits good empathy, showing genuine concern and understanding towards patient's emotions.
5: Extremely empathic, displaying exceptional understanding, compassion, and sensitivity towards patient's emotions, fostering trust and rapport.
""")


Maintaining Patient Welfare

Assessed By Specialist &
Patient Actor

In [ ]:
#To what extent did the doctor maintain the patient's welfare?

#INPUT
dspy.InputField(desc="Criteria: To what extent did the doctor maintain the patient's welfare?")

#OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
0: Cannot rate / Does not apply / Doctor did not perform this
1: Causes patient physical or emotional discomfort AND jeopardizes patient safety
2: Shows minimal regard for patient welfare, with actions that may cause discomfort or minor safety concerns.
3: Demonstrates moderate effort to maintain patient welfare, but with occasional oversights or lapses.
4: Exhibits good consideration for patient welfare, ensuring comfort, safety, and dignity in most situations.
5: Treats patient respectfully and sensitively, prioritizing comfort, safety, and dignity at all times, fostering trust and confidence.
""")


# PCCBP

Fostering the Relationship

Assessed by Specialist

In [ ]:
"""How would you rate the doctor's behavior of FOSTERING A RELATIONSHIP with the patient? On the given criteria. """
dspy.InputField(desc="Conversation between patient and doctor")

In [ ]:
# Build rapport and connection

# INPUT
dspy.InputField(desc="Criteria: Build rapport and connection")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Minimal effort to build rapport and connection, lacks engagement with the patient.
2: Poor - Shows some effort but falls short in establishing rapport and connection with the patient.
3: Fair - Makes moderate attempts to build rapport and connection, with some success in engaging with the patient.
4: Good - Demonstrates effective efforts in building rapport and connection, establishing a positive interaction with the patient.
5: Excellent - Excels in building rapport and connection, establishing a strong and genuine connection with the patient, fostering a trusting relationship.
""")

In [ ]:
# Appear open and honest

# INPUT
dspy.InputField(desc="Criteria: Appear open and honest")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Appears closed off and dishonest, lacking transparency in communication.
2: Poor - Shows some effort but lacks consistency in appearing open and honest.
3: Fair - Makes moderate attempts to appear open and honest, with some sincerity in communication.
4: Good - Demonstrates openness and honesty in communication, establishing trust with the patient.
5: Excellent - Consistently appears open and honest, fostering a transparent and trustworthy relationship with the patient.
""")


In [ ]:
# Discuss mutual roles and responsibilities (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Discuss mutual roles and responsibilities")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to discuss mutual roles and responsibilities, lacks clarity in defining respective roles.
2: Poor - Shows limited effort in discussing mutual roles and responsibilities, with unclear expectations.
3: Fair - Makes some attempt to discuss mutual roles and responsibilities, with moderate clarity in defining respective roles.
4: Good - Demonstrates effective communication about mutual roles and responsibilities, establishing clear expectations.
5: Excellent - Excels in discussing mutual roles and responsibilities, establishing a clear and mutually beneficial partnership.
""")

In [ ]:
# Respect patient statements, privacy, and autonomy (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Respect patient statements, privacy, and autonomy")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Disregards patient statements and autonomy, lacks respect for patient privacy.
2: Poor - Shows some effort but falls short in respecting patient statements, privacy, and autonomy.
3: Fair - Makes moderate attempts to respect patient statements, privacy, and autonomy, with occasional lapses.
4: Good - Demonstrates consistent respect for patient statements, privacy, and autonomy, fostering trust and confidence.
5: Excellent - Consistently respects patient statements, privacy, and autonomy, prioritizing patient dignity and autonomy.
""")

In [ ]:
# Engage in partnership building (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Engage in partnership building")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to engage in partnership building, lacks collaboration with the patient.
2: Poor - Shows limited effort in partnership building, with minimal collaboration with the patient.
3: Fair - Makes some attempt to engage in partnership building, with moderate collaboration with the patient.
4: Good - Demonstrates effective engagement in partnership building, fostering collaboration and shared decision-making.
5: Excellent - Excels in partnership building, actively involving the patient in decision-making and treatment planning.
""")

In [ ]:
# Express caring and commitment (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Express caring and commitment")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to express caring and commitment, lacks empathy and dedication.
2: Poor - Shows minimal effort in expressing caring and commitment, with limited empathy and dedication.
3: Fair - Makes some attempt to express caring and commitment, with moderate empathy and dedication.
4: Good - Demonstrates consistent expression of caring and commitment, fostering a supportive and compassionate relationship.
5: Excellent - Consistently expresses genuine caring and commitment, fostering a strong bond of trust and support.
""")

In [ ]:
# Acknowledge and express sorrow for mistakes (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Acknowledge and express sorrow for mistakes")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to acknowledge or express sorrow for mistakes, lacks accountability and humility.
2: Poor - Shows minimal effort in acknowledging or expressing sorrow for mistakes, with limited accountability and humility.
3: Fair - Makes some attempt to acknowledge or express sorrow for mistakes, with moderate accountability and humility.
4: Good - Demonstrates consistent acknowledgment and expression of sorrow for mistakes, fostering trust and transparency.
5: Excellent - Consistently acknowledges and expresses genuine sorrow for mistakes, demonstrating humility and commitment to improvement.
""")

In [ ]:
# Greet patient appropriately (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Greet patient appropriately")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to greet the patient appropriately, lacks professionalism and courtesy.
2: Poor - Shows minimal effort in greeting the patient appropriately, with occasional lapses in professionalism.
3: Fair - Makes some attempt to greet the patient appropriately, with moderate professionalism and courtesy.
4: Good - Demonstrates consistent professionalism and courtesy in greeting the patient appropriately.
5: Excellent - Consistently greets the patient appropriately with professionalism and warmth, setting a positive tone for interaction.
""")

In [ ]:
# Use appropriate language (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Use appropriate language")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Uses inappropriate language, lacking sensitivity and professionalism.
2: Poor - Shows some effort but occasionally uses inappropriate language, with minor lapses in professionalism.
3: Fair - Makes moderate attempts to use appropriate language, with occasional lapses in sensitivity.
4: Good - Demonstrates consistent use of appropriate language, maintaining professionalism and sensitivity.
5: Excellent - Consistently uses appropriate language, demonstrating professionalism and respect for the patient.
""")

In [ ]:
# Encourage patient participation (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Encourage patient participation")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to encourage patient participation, lacks engagement and involvement.
2: Poor - Shows minimal effort in encouraging patient participation, with limited engagement and involvement.
3: Fair - Makes some attempt to encourage patient participation, with moderate engagement and involvement.
4: Good - Demonstrates consistent encouragement of patient participation, fostering collaboration and involvement.
5: Excellent - Consistently encourages patient participation, actively involving the patient in decision-making and treatment planning.
""")

In [ ]:
# Show interest in the patient as a person (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Show interest in the patient as a person")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to show interest in the patient as a person, lacks empathy and connection.
2: Poor - Shows minimal effort in showing interest in the patient as a person, with limited empathy and connection.
3: Fair - Makes some attempt to show interest in the patient as a person, with moderate empathy and connection.
4: Good - Demonstrates consistent interest in the patient as a person, fostering rapport and understanding.
5: Excellent - Consistently shows genuine interest in the patient as a person, fostering a meaningful and supportive relationship.
""")

Gathering Information

In [ ]:
"""How would you rate the doctor's behavior of GATHERING INFORMATION from the patient?"""
dspy.InputField(desc="Conversation between patient and doctor")

In [ ]:
# Attempt to understand the patient’s needs for the encounter (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Attempt to understand the patient’s needs for the encounter")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to attempt understanding of the patient’s needs, lacking empathy and insight.
2: Poor - Shows minimal effort in attempting to understand the patient’s needs, with limited empathy and insight.
3: Fair - Makes some attempt to understand the patient’s needs, with moderate empathy and insight.
4: Good - Demonstrates consistent effort in attempting to understand the patient’s needs, fostering empathy and insight.
5: Excellent - Consistently shows genuine effort in understanding the patient’s needs, fostering empathy, insight, and patient-centered care.
""")

In [ ]:
# Elicit full description of major reason for visit from biologic and psychosocial perspectives (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Elicit full description of major reason for visit from biologic and psychosocial perspectives")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to elicit a full description of the reason for visit from both biologic and psychosocial perspectives.
2: Poor - Shows minimal effort in eliciting a full description of the reason for visit from both biologic and psychosocial perspectives, with limited insight.
3: Fair - Makes some attempt to elicit a full description of the reason for visit from both biologic and psychosocial perspectives, with moderate insight.
4: Good - Demonstrates consistent effort in eliciting a full description of the reason for visit from both biologic and psychosocial perspectives, fostering understanding and empathy.
5: Excellent - Consistently shows genuine effort in eliciting a comprehensive description of the reason for visit from both biologic and psychosocial perspectives, fostering a patient-centered approach.
""")

In [ ]:
# Ask open-ended questions (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Ask open-ended questions")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to ask open-ended questions, relying on closed-ended inquiries.
2: Poor - Shows minimal effort in asking open-ended questions, with limited exploration.
3: Fair - Makes some attempt to ask open-ended questions, with moderate exploration.
4: Good - Demonstrates consistent use of open-ended questions, fostering comprehensive exploration and understanding.
5: Excellent - Consistently asks open-ended questions, facilitating thorough exploration and patient engagement.
""")

In [ ]:
# Allow patient to complete responses and listen actively (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Allow patient to complete responses and listen actively")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Interrupts or does not allow patient to complete responses, lacks active listening skills.
2: Poor - Shows minimal effort in allowing patient to complete responses and listening actively, with occasional interruptions.
3: Fair - Makes some attempt to allow patient to complete responses and listens actively, with moderate interruptions.
4: Good - Demonstrates consistent practice of allowing patient to complete responses and actively listening, fostering patient-centered communication.
5: Excellent - Consistently allows patient to complete responses and listens actively, demonstrating exceptional communication skills and empathy.
""")


In [ ]:
# Elicit patient’s full set of concerns (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Elicit patient’s full set of concerns")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to elicit patient’s full set of concerns, missing important aspects.
2: Poor - Shows minimal effort in eliciting patient’s full set of concerns, with limited exploration.
3: Fair - Makes some attempt to elicit patient’s full set of concerns, with moderate exploration.
4: Good - Demonstrates consistent effort in eliciting patient’s full set of concerns, fostering understanding and empathy.
5: Excellent - Consistently shows genuine effort in eliciting patient’s full set of concerns, facilitating comprehensive understanding and patient engagement.
""")

In [ ]:
# Elicit patient’s perspective on the problem/illness (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Elicit patient’s perspective on the problem/illness")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to elicit patient’s perspective on the problem/illness, lacks empathy and insight.
2: Poor - Shows minimal effort in eliciting patient’s perspective on the problem/illness, with limited exploration.
3: Fair - Makes some attempt to elicit patient’s perspective on the problem/illness, with moderate exploration.
4: Good - Demonstrates consistent effort in eliciting patient’s perspective on the problem/illness, fostering understanding and empathy.
5: Excellent - Consistently shows genuine effort in eliciting patient’s perspective on the problem/illness, facilitating patient-centered care.
""")

In [ ]:
# Explore full effect of the illness (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Explore full effect of the illness")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to explore the full effect of the illness, lacks understanding of patient’s experience.
2: Poor - Shows minimal effort in exploring the full effect of the illness, with limited insight into patient’s experience.
3: Fair - Makes some attempt to explore the full effect of the illness, with moderate insight into patient’s experience.
4: Good - Demonstrates consistent effort in exploring the full effect of the illness, fostering empathy and understanding.
5: Excellent - Consistently explores the full effect of the illness, demonstrating exceptional empathy and patient-centered care.
""")

In [ ]:
# Clarify and summarize information (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Clarify and summarize information")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to clarify and summarize information, lacks clarity and coherence in communication.
2: Poor - Shows minimal effort in clarifying and summarizing information, with occasional confusion in communication.
3: Fair - Makes some attempt to clarify and summarize information, with moderate clarity and coherence in communication.
4: Good - Demonstrates consistent effort in clarifying and summarizing information, fostering understanding and patient engagement.
5: Excellent - Consistently clarifies and summarizes information effectively, facilitating clear communication and patient comprehension.
""")

In [ ]:
# Enquire about additional concerns (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Enquire about additional concerns")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to enquire about additional concerns, lacks thoroughness in addressing patient’s needs.
2: Poor - Shows minimal effort in enquiring about additional concerns, with limited thoroughness in addressing patient’s needs.
3: Fair - Makes some attempt to enquire about additional concerns, with moderate thoroughness in addressing patient’s needs.
4: Good - Demonstrates consistent effort in enquiring about additional concerns, fostering thoroughness in addressing patient’s needs.
5: Excellent - Consistently enquires about additional concerns, demonstrating exceptional thoroughness in addressing patient’s needs.
""")

Providing Information

In [ ]:
"""How would you rate the doctor's behavior of PROVIDING INFORMATION to the patient?"""
dspy.InputField(desc="Conversation between patient and doctor")

In [ ]:
# Seek to understand patient’s informational needs (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Seek to understand patient’s informational needs")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to seek understanding of patient’s informational needs, lacks insight into patient’s knowledge and concerns.
2: Poor - Shows minimal effort in seeking to understand patient’s informational needs, with limited exploration of patient’s knowledge and concerns.
3: Fair - Makes some attempt to seek understanding of patient’s informational needs, with moderate exploration of patient’s knowledge and concerns.
4: Good - Demonstrates consistent effort in seeking to understand patient’s informational needs, fostering patient engagement and participation.
5: Excellent - Consistently seeks to understand patient’s informational needs effectively, demonstrating exceptional empathy and patient-centered care.
""")


In [ ]:
# Share information (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Share information")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to share information effectively, lacking clarity and coherence in communication.
2: Poor - Shows minimal effort in sharing information, with occasional confusion in communication.
3: Fair - Makes some attempt to share information, with moderate clarity and coherence in communication.
4: Good - Demonstrates consistent effort in sharing information effectively, fostering understanding and patient engagement.
5: Excellent - Consistently shares information effectively, facilitating clear communication and patient comprehension.
""")


In [ ]:
# Overcome barriers to patient understanding (language, health literacy, hearing, numeracy) (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Overcome barriers to patient understanding (language, health literacy, hearing, numeracy)")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to overcome barriers to patient understanding, lacking adaptation to patient needs and limitations.
2: Poor - Shows minimal effort in overcoming barriers to patient understanding, with limited adaptation to patient needs and limitations.
3: Fair - Makes some attempt to overcome barriers to patient understanding, with moderate adaptation to patient needs and limitations.
4: Good - Demonstrates consistent effort in overcoming barriers to patient understanding, fostering inclusivity and accessibility.
5: Excellent - Consistently overcomes barriers to patient understanding effectively, demonstrating exceptional adaptability and patient-centered care.
""")


In [ ]:
# Facilitate understanding (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Facilitate understanding")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to facilitate understanding effectively, lacking clarity and coherence in communication.
2: Poor - Shows minimal effort in facilitating understanding, with occasional confusion in communication.
3: Fair - Makes some attempt to facilitate understanding, with moderate clarity and coherence in communication.
4: Good - Demonstrates consistent effort in facilitating understanding, fostering patient engagement and comprehension.
5: Excellent - Consistently facilitates understanding effectively, ensuring clear communication and patient comprehension.
""")

In [ ]:
# Provide information resources and help patient evaluate and use them (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Provide information resources and help patient evaluate and use them")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to provide information resources and help patient evaluate and use them, lacking support for patient self-education.
2: Poor - Shows minimal effort in providing information resources and helping patient evaluate and use them, with limited support for patient self-education.
3: Fair - Makes some attempt to provide information resources and help patient evaluate and use them, with moderate support for patient self-education.
4: Good - Demonstrates consistent effort in providing information resources and helping patient evaluate and use them, fostering patient empowerment.
5: Excellent - Consistently provides information resources and effectively helps patient evaluate and use them, facilitating patient self-education and empowerment.
""")

In [ ]:
# Explain nature of the problem and approach to diagnosis/treatment (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Explain nature of the problem and approach to diagnosis/treatment")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to explain the nature of the problem and approach to diagnosis/treatment effectively, lacking clarity and coherence in communication.
2: Poor - Shows minimal effort in explaining the nature of the problem and approach to diagnosis/treatment, with occasional confusion in communication.
3: Fair - Makes some attempt to explain the nature of the problem and approach to diagnosis/treatment, with moderate clarity and coherence in communication.
4: Good - Demonstrates consistent effort in explaining the nature of the problem and approach to diagnosis/treatment, fostering patient understanding and engagement.
5: Excellent - Consistently explains the nature of the problem and approach to diagnosis/treatment effectively, ensuring clear communication and patient comprehension.
""")

In [ ]:
# Give uncomplicated explanations and instructions (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Give uncomplicated explanations and instructions")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to give uncomplicated explanations and instructions, using complex language and instructions.
2: Poor - Shows minimal effort in giving uncomplicated explanations and instructions, with occasional use of complex language and instructions.
3: Fair - Makes some attempt to give uncomplicated explanations and instructions, with moderate clarity and simplicity.
4: Good - Demonstrates consistent effort in giving uncomplicated explanations and instructions, using clear and simple language and instructions.
5: Excellent - Consistently gives uncomplicated explanations and instructions effectively, ensuring clarity and understanding.
""")

In [ ]:
# Avoid jargon and complexity (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Avoid jargon and complexity")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to avoid jargon and complexity, using language and terminology that is difficult for the patient to understand.
2: Poor - Shows minimal effort in avoiding jargon and complexity, with occasional use of language and terminology that is difficult for the patient to understand.
3: Fair - Makes some attempt to avoid jargon and complexity, with moderate clarity and simplicity in communication.
4: Good - Demonstrates consistent effort in avoiding jargon and complexity, using language and terminology that is accessible and understandable for the patient.
5: Excellent - Consistently avoids jargon and complexity effectively, ensuring clear and understandable communication.
""")

In [ ]:
# Encourage questions and check understanding (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Encourage questions and check understanding")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to encourage questions and check understanding, lacking engagement and validation of patient comprehension.
2: Poor - Shows minimal effort in encouraging questions and checking understanding, with limited validation of patient comprehension.
3: Fair - Makes some attempt to encourage questions and check understanding, with moderate engagement and validation of patient comprehension.
4: Good - Demonstrates consistent effort in encouraging questions and checking understanding, fostering patient engagement and comprehension.
5: Excellent - Consistently encourages questions and effectively checks understanding, ensuring patient engagement and comprehension.
""")

In [ ]:
# Encourage questions and check understanding (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Encourage questions and check understanding")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to encourage questions and check understanding, lacking engagement and validation of patient comprehension.
2: Poor - Shows minimal effort in encouraging questions and checking understanding, with limited validation of patient comprehension.
3: Fair - Makes some attempt to encourage questions and check understanding, with moderate engagement and validation of patient comprehension.
4: Good - Demonstrates consistent effort in encouraging questions and checking understanding, fostering patient engagement and comprehension.
5: Excellent - Consistently encourages questions and effectively checks understanding, ensuring patient engagement and comprehension.
""")

In [ ]:
# Encourage questions and check understanding (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Encourage questions and check understanding")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to encourage questions and check understanding, lacking engagement and validation of patient comprehension.
2: Poor - Shows minimal effort in encouraging questions and checking understanding, with limited validation of patient comprehension.
3: Fair - Makes some attempt to encourage questions and check understanding, with moderate engagement and validation of patient comprehension.
4: Good - Demonstrates consistent effort in encouraging questions and checking understanding, fostering patient engagement and comprehension.
5: Excellent - Consistently encourages questions and effectively checks understanding, ensuring patient engagement and comprehension.
""")


In [ ]:
# Emphasize key messages (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Emphasize key messages")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to emphasize key messages effectively, lacking clarity and emphasis.
2: Poor - Shows minimal effort in emphasizing key messages, with occasional lack of clarity and emphasis.
3: Fair - Makes some attempt to emphasize key messages, with moderate clarity and emphasis.
4: Good - Demonstrates consistent effort in emphasizing key messages, ensuring clarity and importance.
5: Excellent - Consistently emphasizes key messages effectively, ensuring clear communication and understanding.
""")

Decision Making

In [ ]:
"""How would you rate the doctor's behavior of MAKING DECISIONS with the patient?"""

In [ ]:
# Prepare patient for deliberation and enable decision-making (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Prepare patient for deliberation and enable decision-making")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to prepare patient for deliberation and enable decision-making, lacking involvement and empowerment.
2: Poor - Shows minimal effort in preparing patient for deliberation and enabling decision-making, with limited involvement and empowerment.
3: Fair - Makes some attempt to prepare patient for deliberation and enable decision-making, with moderate involvement and empowerment.
4: Good - Demonstrates consistent effort in preparing patient for deliberation and enabling decision-making, fostering involvement and empowerment.
5: Excellent - Consistently prepares patient for deliberation and enables decision-making effectively, ensuring active involvement and empowerment.
""")


In [ ]:
# Outline collaborative action plan (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Outline collaborative action plan")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to outline collaborative action plan effectively, lacking clarity and coherence.
2: Poor - Shows minimal effort in outlining collaborative action plan, with occasional lack of clarity and coherence.
3: Fair - Makes some attempt to outline collaborative action plan, with moderate clarity and coherence.
4: Good - Demonstrates consistent effort in outlining collaborative action plan, ensuring clarity and coherence.
5: Excellent - Consistently outlines collaborative action plan effectively, ensuring clear communication and understanding.
""")


In [ ]:
# Encourage patient to participate in decision-making (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Encourage patient to participate in decision-making")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to encourage patient to participate in decision-making, lacking engagement and empowerment.
2: Poor - Shows minimal effort in encouraging patient to participate in decision-making, with limited engagement and empowerment.
3: Fair - Makes some attempt to encourage patient to participate in decision-making, with moderate engagement and empowerment.
4: Good - Demonstrates consistent effort in encouraging patient to participate in decision-making, fostering involvement and empowerment.
5: Excellent - Consistently encourages patient to participate in decision-making effectively, ensuring active involvement and empowerment.
""")


In [ ]:
# Outline choices (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Outline choices")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to outline choices effectively, lacking clarity and coherence.
2: Poor - Shows minimal effort in outlining choices, with occasional lack of clarity and coherence.
3: Fair - Makes some attempt to outline choices, with moderate clarity and coherence.
4: Good - Demonstrates consistent effort in outlining choices, ensuring clarity and coherence.
5: Excellent - Consistently outlines choices effectively, ensuring clear communication and understanding.
""")


In [ ]:
# Explore patient’s preferences and understanding (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Explore patient’s preferences and understanding")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to explore patient’s preferences and understanding effectively, lacking engagement and empathy.
2: Poor - Shows minimal effort in exploring patient’s preferences and understanding, with limited engagement and empathy.
3: Fair - Makes some attempt to explore patient’s preferences and understanding, with moderate engagement and empathy.
4: Good - Demonstrates consistent effort in exploring patient’s preferences and understanding, fostering empathy and involvement.
5: Excellent - Consistently explores patient’s preferences and understanding effectively, ensuring active engagement and empathy.
""")


In [ ]:
# Reach agreement (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Reach agreement")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to reach agreement effectively, lacking collaboration and mutual understanding.
2: Poor - Shows minimal effort in reaching agreement, with limited collaboration and mutual understanding.
3: Fair - Makes some attempt to reach agreement, with moderate collaboration and mutual understanding.
4: Good - Demonstrates consistent effort in reaching agreement, ensuring collaboration and mutual understanding.
5: Excellent - Consistently reaches agreement effectively, ensuring clear communication and mutual satisfaction.
""")


In [ ]:
# Identify and enlist resources and support (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Identify and enlist resources and support")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to identify and enlist resources and support effectively, lacking assistance and guidance.
2: Poor - Shows minimal effort in identifying and enlisting resources and support, with limited assistance and guidance.
3: Fair - Makes some attempt to identify and enlist resources and support, with moderate assistance and guidance.
4: Good - Demonstrates consistent effort in identifying and enlisting resources and support, ensuring assistance and guidance.
5: Excellent - Consistently identifies and enlists resources and support effectively, ensuring comprehensive assistance and guidance.
""")


In [ ]:
# Discuss follow-up and plan for unexpected outcomes (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Discuss follow-up and plan for unexpected outcomes")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to discuss follow-up and plan for unexpected outcomes effectively, lacking preparation and foresight.
2: Poor - Shows minimal effort in discussing follow-up and plan for unexpected outcomes, with limited preparation and foresight.
3: Fair - Makes some attempt to discuss follow-up and plan for unexpected outcomes, with moderate preparation and foresight.
4: Good - Demonstrates consistent effort in discussing follow-up and plan for unexpected outcomes, ensuring preparation and foresight.
5: Excellent - Consistently discusses follow-up and plan for unexpected outcomes effectively, ensuring comprehensive preparation and foresight.
""")


Enabling Disease and Treatment-Related Behavior

In [ ]:
"""How would you rate the doctor's behavior of ENABLING DISEASE AND TREATMENTRELATED BEHAVIOR in the patient?"""

In [ ]:
# Assess patient’s interest in and capacity for self-management (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Assess patient’s interest in and capacity for self-management")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to assess patient's interest in and capacity for self-management, showing no effort to understand patient's willingness or ability to participate in managing their own health.
2: Poor - Shows minimal effort in assessing patient's interest in and capacity for self-management, with limited exploration of patient's willingness or ability to participate in managing their own health.
3: Fair - Makes some attempt to assess patient's interest in and capacity for self-management, with moderate exploration of patient's willingness or ability to participate in managing their own health.
4: Good - Demonstrates consistent effort in assessing patient's interest in and capacity for self-management, showing understanding of patient's willingness and ability to participate in managing their own health.
5: Excellent - Consistently assesses patient's interest in and capacity for self-management effectively, demonstrating thorough understanding of patient's willingness and ability to actively engage in managing their own health.
""")


In [ ]:
# Provide advice (information needs, coping skills, strategies for success) (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Provide advice (information needs, coping skills, strategies for success)")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to provide effective advice on information needs, coping skills, and strategies for success, lacking clarity and relevance.
2: Poor - Shows minimal effort in providing advice on information needs, coping skills, and strategies for success, with limited relevance and effectiveness.
3: Fair - Makes some attempt to provide advice on information needs, coping skills, and strategies for success, with moderate relevance and effectiveness.
4: Good - Demonstrates consistent effort in providing effective advice on information needs, coping skills, and strategies for success, showing relevance and helpfulness.
5: Excellent - Consistently provides excellent advice on information needs, coping skills, and strategies for success, demonstrating exceptional relevance and effectiveness in empowering the patient.
""")


In [ ]:
# Agree on next steps (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Agree on next steps")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to agree on next steps effectively, lacking collaboration and mutual understanding.
2: Poor - Shows minimal effort in agreeing on next steps, with limited collaboration and mutual understanding.
3: Fair - Makes some attempt to agree on next steps, with moderate collaboration and mutual understanding.
4: Good - Demonstrates consistent effort in agreeing on next steps, ensuring collaboration and mutual understanding.
5: Excellent - Consistently agrees on next steps effectively, ensuring clear communication and mutual satisfaction.
""")


In [ ]:
# Agree on next steps (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Agree on next steps")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to agree on next steps effectively, lacking collaboration and mutual understanding.
2: Poor - Shows minimal effort in agreeing on next steps, with limited collaboration and mutual understanding.
3: Fair - Makes some attempt to agree on next steps, with moderate collaboration and mutual understanding.
4: Good - Demonstrates consistent effort in agreeing on next steps, ensuring collaboration and mutual understanding.
5: Excellent - Consistently agrees on next steps effectively, ensuring clear communication and mutual satisfaction.
""")

In [ ]:
# Assist patient to optimize autonomy and self-management of their problem (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Assist patient to optimize autonomy and self-management of their problem")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to assist patient in optimizing autonomy and self-management of their problem, lacking support and guidance.
2: Poor - Shows minimal effort in assisting patient to optimize autonomy and self-management of their problem, with limited support and guidance.
3: Fair - Makes some attempt to assist patient in optimizing autonomy and self-management of their problem, with moderate support and guidance.
4: Good - Demonstrates consistent effort in assisting patient to optimize autonomy and self-management of their problem, ensuring support and guidance.
5: Excellent - Assists patient effectively in optimizing autonomy and self-management of their problem, providing comprehensive support and guidance.
""")


In [ ]:
# Arrange for needed support (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Arrange for needed support")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to arrange for needed support effectively, lacking assistance and follow-through.
2: Poor - Shows minimal effort in arranging for needed support, with limited assistance and follow-through.
3: Fair - Makes some attempt to arrange for needed support, with moderate assistance and follow-through.
4: Good - Demonstrates consistent effort in arranging for needed support, ensuring assistance and follow-through.
5: Excellent - Arranges for needed support effectively, providing comprehensive assistance and follow-through.
""")


In [ ]:
# Arrange for needed support (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Arrange for needed support")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to arrange for needed support effectively, lacking assistance and follow-through.
2: Poor - Shows minimal effort in arranging for needed support, with limited assistance and follow-through.
3: Fair - Makes some attempt to arrange for needed support, with moderate assistance and follow-through.
4: Good - Demonstrates consistent effort in arranging for needed support, ensuring assistance and follow-through.
5: Excellent - Arranges for needed support effectively, providing comprehensive assistance and follow-through.
""")


In [ ]:
# Advocate for and assist patient with health system (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Advocate for and assist patient with health system")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to advocate for and assist patient with health system effectively, lacking advocacy and support.
2: Poor - Shows minimal effort in advocating for and assisting patient with health system, with limited advocacy and support.
3: Fair - Makes some attempt to advocate for and assist patient with health system, with moderate advocacy and support.
4: Good - Demonstrates consistent effort in advocating for and assisting patient with health system, ensuring advocacy and support.
5: Excellent - Advocates for and assists patient with health system effectively, providing comprehensive advocacy and support.
""")


In [ ]:
# Assess patient’s readiness to change health behaviours (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Assess patient’s readiness to change health behaviours")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to assess patient’s readiness to change health behaviours effectively, lacking understanding and insight.
2: Poor - Shows minimal effort in assessing patient’s readiness to change health behaviours, with limited understanding and insight.
3: Fair - Makes some attempt to assess patient’s readiness to change health behaviours, with moderate understanding and insight.
4: Good - Demonstrates consistent effort in assessing patient’s readiness to change health behaviours, showing understanding and insight.
5: Excellent - Successfully assesses patient’s readiness to change health behaviours effectively, demonstrating exceptional understanding and insight.
""")


In [ ]:
# Elicit patient’s goals, ideas and decisions (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Elicit patient’s goals, ideas and decisions")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Very Poor - Fails to elicit patient’s goals, ideas, and decisions effectively, lacking engagement and exploration.
2: Poor - Shows minimal effort in eliciting patient’s goals, ideas, and decisions, with limited engagement and exploration.
3: Fair - Makes some attempt to elicit patient’s goals, ideas, and decisions, with moderate engagement and exploration.
4: Good - Demonstrates consistent effort in eliciting patient’s goals, ideas, and decisions, fostering engagement and exploration.
5: Excellent - Consistently elicits patient’s goals, ideas, and decisions effectively, ensuring active engagement and comprehensive exploration.
""")


Responding to Emotions

In [ ]:
"""How would you rate the doctor's behavior of RESPONDING TO EMOTIONS expressed by the patient?"""

In [ ]:
# Facilitate patient expression of emotional consequences of illness (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Facilitate patient expression of emotional consequences of illness")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Struggles to create an environment conducive to discussing emotional impacts, resulting in limited patient engagement.
2: Attempts to encourage patient expression of emotions, but with limited success in fully eliciting emotional responses.
3: Moderately succeeds in fostering patient expression of emotional consequences, leading to some meaningful discussion about feelings.
4: Effectively creates an open atmosphere for patients to share emotional experiences, resulting in substantial discussion and expression of feelings.
5: Excellently facilitates patient expression of emotional impacts, resulting in a deep exploration of emotions and experiences, fostering therapeutic communication.
""")

In [ ]:
# Acknowledge and explore emotions (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Acknowledge and explore emotions")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Rarely acknowledges or explores patient emotions, lacking in empathy and depth of understanding.
2: Makes occasional attempts to acknowledge and explore emotions, but often lacks depth and may miss important emotional cues.
3: Shows moderate skill in acknowledging and exploring emotions, demonstrating some empathy and understanding.
4: Consistently acknowledges and explores patient emotions, showing empathy and fostering a deeper understanding of emotional experiences.
5: Expertly acknowledges and explores emotions, demonstrating exceptional empathy and skill in navigating and validating patient feelings.
""")


In [ ]:
# Express empathy, sympathy, reassurance (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Express empathy, sympathy, reassurance")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Rarely expresses empathy, sympathy, or reassurance, leaving patients feeling unsupported and misunderstood.
2: Occasionally expresses empathy, sympathy, or reassurance, but may lack sincerity or effectiveness in providing comfort.
3: Demonstrates moderate skill in expressing empathy, sympathy, or reassurance, offering some level of comfort and support.
4: Consistently expresses empathy, sympathy, and reassurance, providing meaningful comfort and reassurance to patients.
5: Expertly expresses empathy, sympathy, and reassurance, offering unwavering support and comfort, significantly easing patient distress.
""")


In [ ]:
# Provide help in dealing with emotions (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Provide help in dealing with emotions")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Offers minimal assistance in helping patients deal with emotions, providing little guidance or support.
2: Attempts to provide help in dealing with emotions, but often lacks effectiveness or relevance in the assistance offered.
3: Provides moderate assistance in helping patients deal with emotions, offering some guidance and support.
4: Consistently offers helpful assistance in dealing with emotions, providing relevant guidance and support to patients.
5: Provides exceptional help in dealing with emotions, offering comprehensive guidance and support that significantly aids patients in managing their emotional well-being.
""")


In [ ]:
# Assess psychological distress (1 to 5)

# INPUT
dspy.InputField(desc="Criteria: Assess psychological distress")

# OUTPUT
dspy.OutputField(desc="""Rating based on a 5 point scale for the given criteria:
1: Fails to effectively assess psychological distress, overlooking important emotional aspects of patient well-being.
2: Makes limited attempts to assess psychological distress, but often misses key emotional indicators.
3: Moderately assesses psychological distress, recognizing some emotional aspects but may overlook others.
4: Consistently assesses psychological distress, recognizing and addressing important emotional indicators of patient well-being.
5: Expertly assesses psychological distress, demonstrating exceptional skill in recognizing and addressing a wide range of emotional indicators, leading to a comprehensive understanding of patient emotional well-being.
""")


# Diagnosis and Management